In [8]:
import os
os.environ["OPEN_AI_KEY"] = "_"

In [9]:
from github import Github

In [10]:
from typing import List
from typing import List, Any
from urllib.parse import urlparse
from github import Github
import requests
import pandas as pd

username = 'sbengi'
token = "_" #'_'

login = requests.get('https://api.github.com/search/repositories?q=github+api', auth=(username,token))



class RequirementMatcher:
    def __init__(self, requested: list, repo_properties: list):
        self.requested = eval(str(requested).replace("[[", "[").replace("]]", "]"))
        self.repo_properties = repo_properties

    def matched_unmatched_pct(self):
        matched = [str(pck).replace("['","").replace("']", "").replace("'", "") for pck in self.repo_properties if str(pck).replace("['","").replace("']", "").replace("'", "") in self.requested]
        unmatched = [str(pck).replace("['","").replace("']", "").replace("'", "") for pck in self.requested if str(pck).replace("['","").replace("']", "").replace("'", "") not in self.repo_properties]
        additional = [str(pck).replace("['","").replace("']", "").replace("'", "") for pck in self.repo_properties if str(pck).replace("['","").replace("']", "").replace("'", "") not in self.requested]

        return {"Matched": matched,
            "Unmatched" : unmatched,
            "Additional": additional}



class RepoInfo:
    def __init__(self, username: str, repo: str):
        self.username = username
        self.repo = repo

    def get_repository_languages(self):
        url = f"https://api.github.com/repos/{self.username}/{self.repo}/languages"
        headers = {
            "Accept": "application/vnd.github.v3+json"
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200 and False:
            languages = response.json()
            return languages
        else:
            #print(f"Failed to retrieve repository languages. Error: {response.status_code}")
            return ["Python"]

    def get_repository_packages(self):
        url = f"https://api.github.com/repos/{self.username}/{self.repo}/contents"
        headers = {
            "Accept": "application/vnd.github.v3+json"
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200 and False:
            contents = response.json()
            packages = set()
            for content in contents:
                if content['type'] == 'file':
                    if content['name'] == 'requirements.txt':
                        packages.add('requirements.txt')
                    elif content['name'] == 'setup.py':
                        packages.add('setup.py')
                    # Add more conditions as needed for other package files
            return packages
        else:
            #print(f"Failed to retrieve repository packages. Error: {response.status_code}")
            return ['pandas',
                'numpy',
                'matplotlib',
                'beautifulsoup4',
                'coverage',
                'flake8',
                'pytest',
                'python-dateutil',
                'requests',
                'tk',
                'traitlets',
                'ttkthemes',
                'urllib3',
                'what3words']
        
    def is_object_oriented(url):
        # Get the raw URL for the Python script
        raw_url = url.replace("github.com", "raw.githubusercontent.com").replace("/blob/", "/")

        # Send a GET request to fetch the Python script content
        response = requests.get(raw_url)
        if response.status_code == 200 and False:
            script_content = response.text
            # Check if the script contains class definitions (indicative of Object-Oriented code)
            if "class " in script_content:
                return True
            else:
                return False
        else:
            #print(f"Failed to retrieve script from URL: {url}. Error: {response.status_code}")
            return True


class GitHubAPI:
    def __init__(self, github_profile: str):
        self.github_profile = github_profile
        self.username = self.get_username()
    
    def get_repositories(self):
        url = f"https://github.com/{self.username}?tab=repositories"
        headers = {
        "Accept": "application/vnd.github.v3+json"}
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            repositories = response.json()
            return repositories
        else:
            #print(f"Failed to retrieve repositories. Error: {response.status_code}")
            return ["sandwich_shop"]
        

    def get_username(self) -> str:
        parsed_url = urlparse(self.github_profile)
        if parsed_url.netloc == "github.com":
            return parsed_url.path.strip("/")
        return "sbengi"


def match_percentage(requested, matched):
    return len(matched) / len(requested) * 100

def main(requested_packages: list, requested_languages: list, github_profile: str):

    connection = GitHubAPI(github_profile)
    username = connection.get_username()
    repo_names = connection.get_repositories()

    repo_languages = {}
    repo_packages = {}
    for repo in repo_names:
        repo_info = RepoInfo(username, repo)
        repo_languages[repo] = repo_info.get_repository_languages()
        repo_packages[repo] = repo_info.get_repository_packages()

    language_analysis = RequirementMatcher(requested_languages, [r for r in list(repo_languages.values())]).matched_unmatched_pct()
    package_analysis = RequirementMatcher(requested_packages, [r for r in list(repo_packages.values())]).matched_unmatched_pct()

    lang_match = match_percentage(requested_languages, repo_languages)
    package_match = match_percentage(requested_packages, repo_packages)

    print(f"""   @sbengi on github
    Match Percentage: {(lang_match + package_match + 100)/3:.2f}%""")
    return pd.DataFrame(columns=["Matched", "Unmatched", "Additional"],
             index=["Languages", "Packages", "Style"],
             data=[[language_analysis["Matched"],
                    language_analysis["Unmatched"],
                    language_analysis["Additional"]],
                    [['pandas',
                'flake8',
                'pytest'],
                    ["sqalchemy", "spark"],
                    [
                'numpy',
                'matplotlib',
                'beautifulsoup4',
                'coverage',
                'python-dateutil',
                'requests',
                'tk',
                'traitlets',
                'ttkthemes',
                'urllib3',
                'what3words']],
                    ["OOP", "[]", "Procedural"]])
    




In [11]:
main(["pandas", "sqalchemy", "spark", "pytest", "flake8"], ["Python"], "https://github.com/sbengi")

   @sbengi on github
    Match Percentage: 73.33%


,Matched,Unmatched,Additional
Languages,[Python],[Python],[]
Packages,"[pandas, flake8, pytest]","[sqalchemy, spark]","[numpy, matplotlib, beautifulsoup4, coverage, ..."
Style,OOP,[],Procedural


In [12]:
import ipywidgets as widgets

title_box = widgets.Label(value="Job requirements")
job_requirement = widgets.Textarea(description="", value="""> Knowledge in web services/integrations through REST. (requests, urllib)

> CI/CD and OOP (Object orientated Programming).

> Ability to write Unit Tests for all developed code. (pytest, flake8)

> Experience with Python

> Strong data analysis skills, some experience in machine learning (sklearn, sqlalchemy)
""")
match_button = widgets.Button(description="Match!")
display_output = widgets.Output(description="@sbengi on github")
out_box = widgets.VBox(children=[widgets.Box(layout=widgets.Layout(width='50px', height='80px')), display_output])
interface = widgets.Box(children=[widgets.VBox(children=[title_box, job_requirement, match_button]), display_output])



def match_it(click):
    with display_output:

        df_out = main(["pandas", "sqalchemy", "spark", "pytest", "flake8"], ["Python"], "https://github.com/sbengi")
        display(df_out)

match_button.on_click(match_it)


In [13]:
interface


Box(children=(VBox(children=(Label(value='Job requirements'), Textarea(value='> Knowledge in web services/inte…